###### Nipype Workflow for ProbTrackX2 fdt_path waypath thresholding and normalization
## DG Version 1: Normalization

In [1]:
from nipype import config
import os,glob,sys,shutil
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from nipype.interfaces.fsl import Info
from nipype.interfaces.ants import WarpImageMultiTransform

MNI_template = Info.standard_image('MNI152_T1_1mm_brain.nii.gz')


cfg = dict(execution={'remove_unnecessary_outputs': False,
                      'keep_inputs': True},
           monitoring={'enabled': True,
                       'sample_frequency': 5}
          )
config.update_config(cfg)

# Setup for DataGrabber inputs needed for thresholding

In [2]:
datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
        outfields=['nodif_brain','affDTItoStruct','affStructToMNI','warpStructToMNI',
                   'invWarpStructToMNI','lHypo']),
        name='datasource')
# create a node to obtain the functional images
datasource.inputs.base_directory = "/data/HCP_Data/HCP_BedpostData/"
datasource.inputs.template ='*'
datasource.inputs.sort_filelist = True

datasource.inputs.field_template = dict(
    nodif_brain='%s/T1w/Diffusion/nodif_brain.nii.gz', 
    affDTItoStruct='addlInfo/subject/%s/dtiToStruct0GenericAffine.mat',
    affStructToMNI='addlInfo/subject/%s/structToMNI0GenericAffine.mat',
    warpStructToMNI='addlInfo/subject/%s/structToMNI1Warp.nii.gz',
    invWarpStructToMNI='addlInfo/subject/%s/structToMNI1InverseWarp.nii.gz',
    lHypo='addlInfo/subject/%s/pbxResults/DTI/Human_BasalForebrain_Left_fdt_paths.nii.gz'
    )

datasource.inputs.template_args = dict(
    nodif_brain=[['subject_id']], 
    affDTItoStruct=[['subject_id']],
    affStructToMNI=[['subject_id']],
    warpStructToMNI=[['subject_id']],
    invWarpStructToMNI=[['subject_id']],
    lHypo=[['subject_id']]
    )

In [ ]:
!ls /data

In [ ]:
subjRootDir = "/data/HCP_Data/HCP_BedpostData/"
FULL_SUBJECT_LIST = [x for x in os.listdir(subjRootDir) if os.path.isdir( subjRootDir+'/addlInfo/subject/'+x+'/pbxResults/DTI')]
print(len(FULL_SUBJECT_LIST),"Subjects are potentially available to be processed!")


"""
Setup for Normalized pbx2 results Computational Pipeline
"""
subj_infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),  name="subj_infosource")
#infosource.iterables = ('subject_id', SampleSubjList)
subj_infosource.iterables = ('subject_id', FULL_SUBJECT_LIST[:10])
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

## Threshold and then Normalize images

In [ ]:
# Node: Function: Thresholds using FSL Maths
# ifiles: original input file
Threshold = pe.Node(fsl.Threshold(),
                 name = 'Threshold')

# Node: Function: Normalizes using FSL Maths -div
Normalize = pe.Node(fsl.BinaryMaths(operation='div'),
                 name = 'Normalize')

In [ ]:
### Had a default but we deleted it for now
### TScaling was = 100000
def getWaytotal(in_file, Tscaling): 
    import subprocess
    import re
    
    in_file= re.sub('fdt_paths.nii.gz', 'waytotal', in_file)
    mycommand= "cat "+str(in_file)
    mycommand= mycommand.split(sep=' ')
    result= subprocess.run(mycommand, stdout=subprocess.PIPE)
    tmpwaytotal= result.stdout.decode("utf-8").split(' ')[0]
    print('waytotal: %s' % tmpwaytotal)
    
    if(not tmpwaytotal): 
        print('Waytotal Empty for: %s' % pathwaytotal)
    else:
        waytotals= float(tmpwaytotal) / Tscaling                
        return( waytotals )
    
    
from nipype.interfaces.utility import Function
getWaytotal_node = pe.Node( name="getWaytotal_node",iterfield=["Tscaling"],
                           interface=Function(input_names=["in_file","Tscaling"],
                            output_names=["waytotal"],
                             function=getWaytotal)
                          )

getWaytotal_node.iterables = ("Tscaling",  [5000,1000,100,50])

In [ ]:
## Register thresholded results to the MNI Space for further analysis
###WARPING PBX RESULTS TO MNI SPACE--- THIS IS THE SAME WAY WE COMPUTED THE ACTUAL TRANSFORMS
#an@frink:/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/testThreshold$ WarpImageMultiTransform 3 nodif_brain.nii.gz nodif_to_mni_debug_warp_strucToMNI_dtiToStruct.nii.gz  -R /usr/share/fsl/5.0/data/standard/MNI152_T1_1mm.nii.gz --use-NN ./structToMNI1Warp.nii.gz ./structToMNI0GenericAffine.mat ./dtiToStruct0GenericAffine.mat 

warp_pbxDTI_to_MNI = pe.Node( WarpImageMultiTransform(use_nearest=True,reference_image=MNI_template), 
                             name="warp_pbxDTI_to_MNI")

## Create a merge node to store the XFMS
merge_xfms = pe.Node(util.Merge(3), name='merge_xfms')

In [ ]:
###  I NEED THE WAYTOTAL AND THE SCALED WAYTOTAL.... OOPS!!!


# Workflow: Initialize
wf = pe.Workflow(name="threshAndWarpPBX")
wf.base_dir = '/data/HCP_Data/NipypeScratch/'
wf.connect(subj_infosource,'subject_id',datasource,'subject_id')

# ### Connect the dti_datasource to the pbx2 command
wf.connect( datasource,'lHypo',Threshold,'in_file')
wf.connect( datasource,'lHypo',getWaytotal_node,'in_file')

wf.connect(getWaytotal_node,'waytotal',Threshold,'thresh')
wf.connect( Threshold,'out_file',Normalize,'in_file')
wf.connect(getWaytotal_node,'waytotal',Normalize,'operand_value')


wf.connect(datasource,'warpStructToMNI',merge_xfms,"in1")
wf.connect(datasource,'affStructToMNI',merge_xfms,"in2")
wf.connect(datasource,'affDTItoStruct',merge_xfms,"in3")

## Connect the pbx2 results I want to warp
#wf.connect(datasource,"lHypo",warp_pbxDTI_to_MNI,"input_image")
wf.connect(Normalize,"out_file",warp_pbxDTI_to_MNI,"input_image")
wf.connect( merge_xfms,  'out', warp_pbxDTI_to_MNI, 'transformation_series')

### need to make sure I feed the same modified waytotal to both functions
wf.run()
# Run Workflow
#wf.run()
# wf.run(plugin='MultiProc', plugin_args={'n_procs': 10})

In [ ]:
!ls /data

In [ ]:
#WarpImageMultiTransform 3 /usr/share/fsl/5.0/data/standard/MNI152_T1_1mm.nii.gz mni_to_dti.nii.gz -R /HCP_Data/HCP_BedpostData/162228/T1w/Diffusion/nodif_brain.nii.gz --use-NN
#./structToMNI1InverseWarp.nii.gz -i 
#./structToMNI0GenericAffine.mat -i ./dtiToStruct0GenericAffine.mat


## Initialize Workflows:

## Connect Nodes:

In [ ]:
def cleanOutput(x):
    import re
    
    x= re.search('_id_(.*?).._seed_..data.*EHECHT_ROIS..(.*?)_(.*?)_(.*?).nii.gz', x)
    x= '%s_%s_%s_%s' % (x.group(1), x.group(2), x.group(3), x.group(4))
    return(x)

In [ ]:
# Connect Nodes
wf.connect(infosource, "fdt_paths", Normalize, "in_file")
wf.connect(infosource, ("fdt_paths", getWaytotal), Normalize, "operand_value")
wf.connect(Normalize, "out_file", Threshold, "in_file")

#wf.connect(Threshold, ("out_file", cleanOutput), Sink, "container")
#wf.connect(Threshold, "out_file", Sink, "container.results")

wf.connect(Threshold, ("out_file", cleanOutput), Sink, "base_directory")
wf.connect(Threshold, "out_file", Sink, "container")

## Write Graph:

In [ ]:
# Workflow: Graph: Exec
wf.write_graph(graph2use='exec', dotfilename='/output/graph_exec.dot')
#wf.write_graph(graph2use='hierarchical', dotfilename='/output/graph_exec.dot')

# Visualize graph
Image(filename="/output/graph_exec.png")

## Print final directory structure:

In [ ]:
%%bash 
# Print directory structure
tree -C -I "*.nii.gz" /data/SASRAID/ThreshNipype/output | grep -v -e ".*report" -e ".*pklz" -e ".*json"

## Generate subject file lists for each feature in group:

In [ ]:
metaloc= '/code/notebooks/postPBX/unrestricted_dagutman_7_12_2018_15_39_53.csv'
group='Gender'
feature='M'
bedpostx_dir= '/data/NipypeScratch/runpbx2/*'

#### bedpostx_results= [subject_directories]

In [ ]:
bedpostx_results= [ fn for fn in glob.glob(bedpostx_dir) if re.search( '[0-9][0-9]$', fn)  ]

### subjectinfo= {subject_id: {'waytotals': [paths], 'fdt_paths': [paths]}}

In [ ]:
subjectinfo= {}
for i in bedpostx_results:
    output_sid= re.search('_id_(.*)', i).group(1)
    waytotals= glob.glob( i+'/_seed*/pbx2/waytotal' )
    fdt_paths= glob.glob( i+'/_seed*/pbx2/fdt_paths.nii.gz' )
    subjectinfo[output_sid]={}
    subjectinfo[output_sid]['waytotals']= waytotals
    subjectinfo[output_sid]['fdt_paths']= fdt_paths

In [ ]:
subjectinfo

In [ ]:
metadata= pd.read_csv(metaloc,header=0)
metadata_group= finalmetadata.groupby(by=group)
metadata_feature= list( metadata_group.get_group(feature)['FDT_Paths'] )

### Merge Males
### Merge Females

## Testing:

In [ ]:
!ls /data/NipypeScratch/runpbx2/_subject_id_880157/_seed*/pbx2/waytotal

In [ ]:
A= '/data/NipypeScratch/runpbx2/_subject_id_880157/_seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz/pbx2/waytotal'

In [ ]:
mycommand= "cat "+A
mycommand= mycommand.split(sep=' ')
result= subprocess.run(mycommand, stdout=subprocess.PIPE)
int( result.stdout )

In [ ]:
! cat '/data/NipypeScratch/runpbx2/_subject_id_880157/_seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz/pbx2/waytotal'

In [ ]:
! ls /NipypeScratch

In [ ]:
infosource = Node(IdentityInterface(fields=['fdt_path']),
                  name="infosource")
infosource.iterables = [('fdt_path', fdt_paths)]

In [ ]:

## STEPS 



### GET WAYTOTAL FOR the TRACTOGRAPHY RUN
## calculate the threshold which is   0.1% or 0.01%  i.e. 0.001 and 0.0001 * waytotal
### fslmaths fdt_paths.nii.gz -thr <somenumber> -div waytotal (or waytotal/1000 so numbers are not 00000000)


## THEN CONVERT RESULTS TO MNI SPACE---   AAH CHA!! OK


## THEN ADD EM UP AND MAKE A MAP...










In [ ]:
from os.path import abspath
from IPython.display import Image

from nipype import SelectFiles, Node, MapNode, Workflow, Function
from nipype.interfaces import fsl
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import ImageStats
from nipype.interfaces.io import DataSink
from nipype.interfaces.utility import IdentityInterface

import networkx
import graphviz as gv
import os
import base64
import re
import glob
import sys
import subprocess

import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from plotly import __version__
from plotly.offline import init_notebook_mode, plot, iplot
from plotly import graph_objs as go

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=False)